# Transfer Learning in PyTorch

This notebook shows how to use Transfer Learning in PyTorch. 

We will focus on Computer Vision tasks using CalTech 101.

# 0. Imports, Constants and Support Functions

In [1]:
import torch
from torchvision import datasets, transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# 1. Load and preprocess the dataset

In [2]:
# Define image transformations
transform = transforms.Compose(
    [
        transforms.Lambda(lambda image: image.convert("RGB")),
        transforms.Resize(256),  # 256x256
        transforms.CenterCrop(224),  # 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [3]:
# Download dataset
dataset = datasets.Caltech101(root="./data", download=True, transform=transform)
# Split between train and test
train_set, test_set = train_test_split(dataset, test_size=0.1, random_state=14)

Files already downloaded and verified


In [4]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

# 2. Load and use pre-trained models

In [5]:
# Load pretrained model
pretrained_model = models.resnet18(pretrained=True)

/Users/miguelcachosoblechero/.pyenv/versions/3.11.5/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/miguelcachosoblechero/.pyenv/versions/3.11.5/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
for param in pretrained_model.parameters():
    # Establish that the model doesn't need gradient
    # as it is not going to be trained
    param.requires_grad = False

In [7]:
# Modify the last layer
pretrained_model.fc = torch.nn.Linear(pretrained_model.fc.in_features, 101)

In [8]:
# Define devices
device = torch.device("cpu")
pretrained_model.to(device)

# Define training parameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    pretrained_model.fc.parameters(), lr=0.001
)  # Only last layer
num_epochs = 10

In [11]:
# Set the model in training mode
pretrained_model.train()

# Define training loop
for epoch in range(num_epochs):
    # Start logging on each iteration
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader):
        # Move to device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        prediction = pretrained_model(inputs)

        # Calculate loss
        loss = criterion(prediction, labels)

        # Reset gradient after each iteration
        optimizer.zero_grad()  # This needs to be done on each batch!

        # Calculate gradients
        loss.backward()

        # Update parameters
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

100%|██████████| 245/245 [03:32<00:00,  1.15it/s]


Epoch [1/10], Loss: 0.0084


  1%|          | 2/245 [00:02<04:08,  1.02s/it]


KeyboardInterrupt: 